In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import warnings

file_path = "filtered_data.xlsx"
data = pd.read_excel(file_path)

print("数据基本信息:")
print(data.info())
print("\n数据预览:")
print(data.head())

# 定义二分类字段列表
binary_vars = [
    '性别', '冠心病', '高血压', '糖尿病', '心房颤动', '肝硬化', '慢性肾脏系统疾病',
    '呼吸系统疾病', '脑卒中', '其他系统肿瘤', '肝脏肿瘤', '胃肠道肿瘤', '阿司匹林', '氯吡格雷',
    '利伐沙班', '华法林', '糖皮质激素', '非甾体抗炎药', '其他用药', '既往手术史',
    '呕血', '黑便', '便血', '呕咖啡色胃内容物', '胸痛', '晕厥', '四肢冰冷', 
    '肌酸激酶同工酶高于正常高限', '肌红蛋白高于正常高限', '肌钙蛋白高于正常高限',
    'PPI(808方案）', '生长抑素', '是否胃镜', '硬化术', '电凝术', '注射药物', '套扎术', 
    '钛夹', '抗生素', '血管活性药物', '是否输血'
]

# 应用One-Hot编码到二分类变量
# 男性=1, 女性=0; 存在=1, 不存在=0; 是=1, 否=0
for col in binary_vars:
    if col in data.columns:
        data[col] = data[col].astype('category')
        top_categories = data[col].value_counts().index[:2]
        if len(top_categories) >= 1:
            main_category = top_categories[0]
            data[col] = data[col].apply(lambda x: 1 if x == main_category else 0)
        else:
            # 如果只有一个类别，全部设为1
            data[col] = 1

categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
# 移除已经是二分类的变量
categorical_cols = [col for col in categorical_cols if col not in binary_vars]
# 使用One-Hot编码处理多分类变量
if categorical_cols:
    # 创建列转换器
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ],
        remainder='passthrough'
    )
    # 应用转换
    transformed_data = preprocessor.fit_transform(data)
    # 获取新的列名
    new_columns = []
    if preprocessor.named_transformers_['cat'] is not None:
        ohe_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
        new_columns.extend(ohe_columns)
    
    # 添加非分类列
    non_categorical_cols = [col for col in data.columns if col not in categorical_cols]
    new_columns.extend(non_categorical_cols)
    
    # 创建新的DataFrame
    data = pd.DataFrame(transformed_data, columns=new_columns)

#  整数变量插补（中位数）
int_cols = ['p', 'R']
for col in int_cols:
    if col in data.columns:
        # 检查是否有缺失值
        if data[col].isnull().sum() > 0:
            print(f"对整数变量 {col} 使用中位数插补...")
            median_val = data[col].median()
            data[col].fillna(median_val, inplace=True)

# 3.2 连续变量插补（随机森林）
cont_cols = [ 
    '收缩压', '舒张压', '休克指数', '平均动脉压', 'SpO2',
    '天门冬氨酸转移酶', '丙氨酸转氨酶', '碱性磷酸酶', '谷氨酰胺转肽酶', 
    '总胆红素', '总蛋白', '白蛋白', '血糖', '血钾',
    '活化部分凝血活酶时间', '凝血酶原百分活动度', '国际标准化比值',
    'HCT', 'BuN', 'Hb', '血肌酐', '血小板',
    '建立静脉通路所需时限', '急诊胃镜完成时限', '门镜时间', '来院采血时限'
]

# 仅处理数据中存在的列
cont_cols = [col for col in cont_cols if col in data.columns]

# 创建随机森林插补器
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    max_iter=10,
    random_state=42
)

# 应用随机森林插补
print("对连续变量使用随机森林插补...")
cont_data = data[cont_cols].copy()
cont_data_imputed = rf_imputer.fit_transform(cont_data)
data[cont_cols] = cont_data_imputed

# 二分类变量插补（MICE + 逻辑回归）
# 需要插补的二分类变量
missing_binary_vars = [
    '黑便', '便血', '呕咖啡色胃内容物', '肌酸激酶同工酶高于正常高限', 
    '肌红蛋白高于正常高限', '肌钙蛋白高于正常高限', '抗生素', '血管活性药物'
]

# 仅处理数据中存在的列
missing_binary_vars = [col for col in missing_binary_vars if col in data.columns]

# 创建MICE插补器（使用逻辑回归）
mice_imputer = IterativeImputer(
    estimator=LogisticRegression(max_iter=1000),
    max_iter=10,
    random_state=42
)

# 应用MICE插补
print("对二分类变量使用MICE插补...")
binary_data = data[missing_binary_vars].copy()
binary_data_imputed = mice_imputer.fit_transform(binary_data)

# 将概率转换为0/1 (阈值=0.5)
binary_data_imputed = np.where(binary_data_imputed > 0.5, 1, 0)
data[missing_binary_vars] = binary_data_imputed

#  检查插补结果
# 检查插补后缺失值
missing_after = data.isnull().sum()
print("\n插补后缺失值概况：")
print(missing_after[missing_after > 0])

# 保存处理后的数据
output_path = "processed_data.xlsx"
data.to_excel(output_path, index=False)